In [ ]:
import warnings 
warnings.simplefilter('ignore')
import pandas as pd
pd.set_option('display.max_columns', 30)

In [ ]:
import numpy as np
SEED = 42
np.random.seed(SEED)

In [ ]:
de_train = pd.read_parquet('/kaggle/input/open-problems-single-cell-perturbations/de_train.parquet')

In [ ]:
print(de_train.shape)

In [ ]:
de_train

In [ ]:
id_map = pd.read_csv('/kaggle/input/open-problems-single-cell-perturbations/id_map.csv')

In [ ]:
id_map

In [ ]:
sub_0_566 = pd.read_csv('/kaggle/input/op-scp-submissions-3/submission_0_566.csv')

In [ ]:
sub_0_566 = pd.concat([id_map, sub_0_566], axis = 1).drop(columns = 'id')
sub_0_566

In [ ]:
de_train = pd.concat([de_train, sub_0_566, sub_0_566, sub_0_566], ignore_index = True)
de_train

In [ ]:
Y = de_train.iloc[:, 5:]

In [ ]:
cat_features = ['cell_type', 'sm_name']

In [ ]:
dummies_train = pd.get_dummies(de_train[cat_features], columns=cat_features)

In [ ]:
print(dummies_train.shape)


In [ ]:
dummies_train

In [ ]:
dummies_test = pd.get_dummies(id_map[cat_features], columns=cat_features)

In [ ]:
print(dummies_test.shape)

In [ ]:
dummies_test

In [ ]:
drop_feats = [feat for feat in dummies_train if feat not in dummies_test]
drop_feats

In [ ]:
X = dummies_train.drop(columns = drop_feats)
X.shape[1], dummies_test.shape[1]

In [ ]:
X

In [ ]:
cell_type_tr = de_train.drop(columns = ['sm_name', 'sm_lincs_id', 'SMILES', 'control'])
cell_type_tr

In [ ]:
sm_name_tr = de_train.drop(columns = ['cell_type', 'sm_lincs_id', 'SMILES', 'control'])
sm_name_tr

In [ ]:
cell_type_te = sub_0_566.drop(columns = ['sm_name'])
cell_type_te

In [ ]:
sm_name_te = sub_0_566.drop(columns = ['cell_type'])

In [ ]:
sm_name_te

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR

In [ ]:
model_knr = KNeighborsRegressor(n_neighbors=13)
model_lsvr = LinearSVR(max_iter = 2000, epsilon = 0.1)

In [ ]:
from tqdm import tqdm
Y_preds = []
for gene in tqdm(range(Y.shape[1])):
    Y_gene = Y.iloc[:, gene].copy()
    X_gene = X.join(cell_type_tr.iloc[:, gene+1]).copy()
    X_gene = X_gene.join(sm_name_tr.iloc[:, gene+1], lsuffix='_cell_type', rsuffix = '_sm_name' )
    test_gene = dummies_test.join(cell_type_te.iloc[:, gene+1]).copy()
    test_gene = test_gene.join(sm_name_te.iloc[:, gene+1], lsuffix='_cell_type', rsuffix = '_sm_name')
    model_knr.fit(X_gene, Y_gene)
    model_lsvr.fit(X_gene, Y_gene)
    Y_pred_knr = model_knr.predict(test_gene)
    Y_pred_lsvr = model_lsvr.predict(test_gene)
    Y_preds.append(Y_pred_knr*0.3+Y_pred_lsvr*0.7)

In [ ]:
submit = pd.DataFrame(Y_preds).T
submit.columns = de_train.columns[5:]
sumit.index.name = 'id'
submit

In [ ]:
submit.to_csv('submission_1.csv')

In [ ]:
data = pd.read_csv('/kaggle/working/submission_1.csv')

In [ ]:
data

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'')